In [1]:
import scipy.io
import torch
import sys
import os
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

In [2]:
mat_folder_path = 'C:\\Users\\ai\\Desktop\\RA 2023\\RNN'
mat_file_name = 'dyn_4g_2023.mat'

# Check if the folder path exists
if not os.path.exists(mat_folder_path):
    print("Folder path does not exist.")
else:
    # Construct the full path to the .mat file
    mat_file_path = os.path.join(mat_folder_path, mat_file_name)
    
    # Check if the file exists
    if not os.path.exists(mat_file_path):
        print("MAT file does not exist in the specified folder.")
    else:
        # Load the .mat file
        mat_data = scipy.io.loadmat(mat_file_path)
        print("MAT file loaded successfully.")


MAT file loaded successfully.


In [3]:
train_feature = mat_data['bearing_pos']   
train_target = mat_data['measured_output']   

In [4]:
# Convert the numpy arrays to PyTorch tensors
train_features_tensor = torch.tensor(train_feature, dtype=torch.float32)
train_target_tensor = torch.tensor(train_target, dtype=torch.float32)

In [5]:
mean = train_features_tensor.mean(dim=0)
std = train_features_tensor.std(dim=0)
# Apply z-score normalization to features_tensor
train_features_tensor = (train_features_tensor - mean) / std

In [6]:
class LSTMRegressionModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMRegressionModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out)  # Output for each time step
        
        return out

In [7]:
# Hyperparameters
input_size = 4  # Number of features
hidden_size = 64
num_layers = 2
output_size = 4  # Regression output (4 targets)

# Reshape features and targets to (batch_size, sequence_length, input_size) format
sequence_length = train_features_tensor.size(1)
train_features_tensor = train_features_tensor.unsqueeze(0)  # Add batch dimension
train_target_tensor = train_target_tensor.unsqueeze(0)  # Add batch dimension

# Create the model
model = LSTMRegressionModel(input_size, hidden_size, num_layers, output_size)

# Loss and optimizer
criterion = nn.MSELoss()  # Mean Squared Error loss for regression
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 1000
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(train_features_tensor)
    loss = torch.sqrt(criterion(outputs, train_target_tensor))   # RMSE
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Make predictions
with torch.no_grad():
    predicted = model(train_features_tensor)

# Print some predictions
print("Predicted:")
print(predicted[:, -1, :])  # Print predictions for the last time step
print("Actual:")
print(train_target_tensor[:, -1, :])  # Print actual targets for the last time step

Epoch [100/1000], Loss: 0.5773
Epoch [200/1000], Loss: 0.5011
Epoch [300/1000], Loss: 0.4778
Epoch [400/1000], Loss: 0.4633
Epoch [500/1000], Loss: 0.4445
Epoch [600/1000], Loss: 0.4280
Epoch [700/1000], Loss: 0.4163
Epoch [800/1000], Loss: 0.3996
Epoch [900/1000], Loss: 0.4081
Epoch [1000/1000], Loss: 0.3805
Predicted:
tensor([[-4.7815, -1.8053, -4.4789, -2.3638]])
Actual:
tensor([[-4.9548, -1.4779, -4.2380, -2.4794]])


In [8]:
mat_folder_path = 'C:\\Users\\ai\\Desktop\\RA 2023\\RNN'
mat_file_name = 'dyn_6g_2023.mat'

# Check if the folder path exists
if not os.path.exists(mat_folder_path):
    print("Folder path does not exist.")
else:
    # Construct the full path to the .mat file
    mat_file_path = os.path.join(mat_folder_path, mat_file_name)
    
    # Check if the file exists
    if not os.path.exists(mat_file_path):
        print("MAT file does not exist in the specified folder.")
    else:
        # Load the .mat file
        mat_data = scipy.io.loadmat(mat_file_path)
        print("MAT file loaded successfully.")

test_feature = mat_data['bearing_pos']   
test_target = mat_data['measured_output'] 

test_features_tensor = torch.tensor(test_feature, dtype=torch.float32)
test_target_tensor = torch.tensor(test_target, dtype=torch.float32)

mean1 = test_features_tensor.mean(dim=0)
std1 = test_features_tensor.std(dim=0)
test_features_tensor = (test_features_tensor - mean1) / std1

MAT file loaded successfully.


In [9]:
# Convert the test features tensor and target tensor to the required shape (batch_size, sequence_length, input_size)
test_features_tensor = test_features_tensor.unsqueeze(0)  # Add batch dimension
test_target_tensor = test_target_tensor.unsqueeze(0)      # Add batch dimension

# Make predictions on the test data
with torch.no_grad():
    test_predictions = model(test_features_tensor)

# Calculate the testing loss
test_loss = torch.sqrt(nn.MSELoss()(test_predictions, test_target_tensor))
print(f"Testing Loss: {test_loss.item():.4f}")

Testing Loss: 0.4966
